In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from dateutil.parser import parse
import urllib.request
from bs4 import BeautifulSoup

In [298]:
standard_schema =  ['ride_id', '' 'duration', 'start_time', 'end_time',
                    'start_station_name', 'end_station_name', 'ridertype', 'bike_system']

pogoh_transforms = "['PO-' + x['ride_id'],int(x['Duration']), parse(str(x['Start Date'])),parse(str(x['End Date'])),x['Start Station Name'],x['End Station Name'],x['Rider Type'],'POGOH']"


healthyride_transforms = "['HR-' + x['ride_id'], int(x['Tripduration']), parse(x['Starttime']),parse(x['Stoptime']),x['From station name'],x['To station name'],x['Usertype'],'Healthy Ride']"

In [255]:
pd.read_csv('https://data.wprdc.org/dataset/bfbaa051-677c-445f-95c5-f9ca129accd4/resource/f8e0d66b-3958-4c8c-97d0-4351b9dfb222/download/healthy-ride-rentals-2018-q1.csv')

,Trip id,Starttime,Stoptime,Bikeid,Tripduration,From station id,From station name,To station id,To station name,Usertype
0,41489490,1/1/2018 17:19,1/1/2018 17:24,70259,276,1022,Liberty Ave & S Millvale Ave (West Penn Hospital),1023,Liberty Ave & Baum Blvd,Subscriber
1,41491355,1/2/2018 7:28,1/2/2018 10:25,70213,10575,1040,O'Hara St and University Place (Soldiers and S...,1037,Frew St & Schenley Dr,Subscriber
2,41491375,1/2/2018 7:32,1/2/2018 7:59,70114,1640,1049,S 12th St & E Carson St,1049,S 12th St & E Carson St,Subscriber
3,41491450,1/2/2018 7:45,1/2/2018 10:18,70252,9172,1002,Third Ave & Wood St,1009,12th St & Penn Ave,Subscriber
4,41492051,1/2/2018 9:09,1/2/2018 9:10,70481,61,1048,S 18th St & Sidney St,1048,S 18th St & Sidney St,Subscriber
...,...,...,...,...,...,...,...,...,...,...
5181,42771071,3/31/2018 23:05,3/31/2018 23:09,70116,198,1008,Centre Ave & PPG Paints Arena,1006,Ross St & Sixth Ave (Steel Plaza T Station),Subscriber
5182,42771073,3/31/2018 23:06,3/31/2018 23:21,70372,866,1033,Ivy St & Walnut St,1020,42nd & Penn Ave.,Customer
5183,42771083,3/31/2018 23:10,3/31/2018 23:14,70298,218,1033,Ivy St & Walnut St,1032,Walnut St & College St,Subscriber
5184,42771105,3/31/2018 23:19,3/31/2018 23:30,70301,707,1043,S Millvale Ave & Centre Ave,1031,Maryland Ave & Ellsworth Ave,Subscriber


In [299]:
def get_trips(url, transforms):
    if url[-4:] == 'xlsx':
        df = pd.read_excel(url)
        df['ride_id'] = [url.split('/')[-1][:8] + '-' + str(x) for x in range(len(df))]
    elif url[-3:] == 'csv':
        df = pd.read_csv(url)
        df['ride_id'] = [url[-10:-3] + '-' + str(x) for x in range(len(df))]
    return df.apply(lambda x: pd.Series(eval(transforms), index = standard_schema), axis = 1)

In [305]:
def get_urls(main_url):
    resource_ids = set()
    page = requests.get(main_url)
    soup = BeautifulSoup(page.content, "html.parser")
    for link in soup.find_all('a'):
        href = link.get('href')
        if href[:31] == 'https://data.wprdc.org/dataset/':
            resource_ids.add(href)
    return list(resource_ids)


pogoh_df = pd.DataFrame()

for resource in get_urls('https://data.wprdc.org/dataset/pogoh-trip-data'):
    pogoh_df = pd.concat([pogoh_df, get_trips(resource, pogoh_transforms)], ignore_index=True)
    print('finished ', resource, len(pogoh_df), '\n \n')

'''
for resource in get_urls('https://data.wprdc.org/dataset/healthyride-trip-data'):
    try:
        pogoh_df = pd.concat([pogoh_df, get_trips(resource, healthyride_transforms)], ignore_index=True)
    except:
        print('Failed to scrape ' + resource)
    print('finished ', resource, len(pogoh_df), '\n \n')
'''

finished  https://data.wprdc.org/dataset/f376ccbc-c851-4b58-8b59-5dd9edc736ee/resource/8e8a5cd9-943e-4d21-a7ed-05f865dd0038/download/apr-2023.xlsx 11351 
 

finished  https://data.wprdc.org/dataset/f376ccbc-c851-4b58-8b59-5dd9edc736ee/resource/1744cd4a-13cf-46de-b773-57f528ee9705/download/july-2022.xlsx 20427 
 

finished  https://data.wprdc.org/dataset/f376ccbc-c851-4b58-8b59-5dd9edc736ee/resource/2cd57068-5498-4b47-b2d3-512d3cdb1e0c/download/may-2022.xlsx 26546 
 

finished  https://data.wprdc.org/dataset/f376ccbc-c851-4b58-8b59-5dd9edc736ee/resource/e2c446b7-b0f5-43d5-87ac-f4b252f27aa3/download/july-2023.xlsx 44792 
 

finished  https://data.wprdc.org/dataset/f376ccbc-c851-4b58-8b59-5dd9edc736ee/resource/4574cdf1-e65a-42df-8ede-eadd4548660a/download/november-2023.xlsx 70595 
 

finished  https://data.wprdc.org/dataset/f376ccbc-c851-4b58-8b59-5dd9edc736ee/resource/a4598299-e02a-43a3-901f-dff08630e96e/download/may-2024.xlsx 102341 
 

finished  https://data.wprdc.org/dataset/f376ccbc-

"\nfor resource in get_urls('https://data.wprdc.org/dataset/healthyride-trip-data'):\n    try:\n        pogoh_df = pd.concat([pogoh_df, get_trips(resource, healthyride_transforms)], ignore_index=True)\n    except:\n        print('Failed to scrape ' + resource)\n    print('finished ', resource, len(pogoh_df), '\n \n')\n"

In [307]:
len(pogoh_df)

df.to_csv('pogoh_trips.tsv', sep="\t") 